# Preparing data

This notebook reads data processed by Galaxy and

 1. Validates sites to ensire that coordinates are correct
 2. Adds information about variants of concern (VOC) and sites under selection


In [74]:
import pandas as pd

In [75]:
!pip install biopython pandasql

In [76]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [77]:
funclass_translation = {'SILENT':'Synonymous','MISSENSE':'Non-synonymous','NONSENSE':'Stop','.':'Non-coding','NONE':'Non-coding'}

## Which dataset to run notebook on?
At this time there are three possible datasets:

 - Boston: `bos`
 - COG-Pre: `cog-pre`
 - COG-Post: `cog-post`

Setting this variable runs all notebook content for this particular dataset. The actual paths are pulled out from `datasets` dict (next cell)

In [78]:
dataset = 'cog-post'

In [79]:
datasets = {
    'bos':
        [   
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz',
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_var.tsv.gz'
        ],
    'cog-pre': 
        [   
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20200917_by_sample.tsv.gz',
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20200917_by_var.tsv.gz'
        ],
    'cog-post': 
        [   
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20201120_by_sample.tsv.gz',
            'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/cog_20201120_by_var.tsv.gz'
        ]
}

In [80]:
# SARS-CoV-2 genome assembly url
gnm_url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.gbff.gz'
gnm_file = gnm_url.split('/')[-1]

# VOC data
voc_url = 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/voc/voc.tsv.gz'

# Selection data
sel_url = 'https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/selection/selection.tsv.gz'

In [81]:
# Get SARS-CoV-2 RefSeq genomes (in GenBank format) from NCBI
import os.path
from os import path
if not path.exists(gnm_file):
    !wget {gnm_url}
    !gunzip {gnm_file}
else:
    print('File {} is already here\nDoing nothing!'.format(gnm_file))

File GCF_009858895.2_ASM985889v3_genomic.gbff.gz is already here
Doing nothing!


In [82]:
from Bio import SeqIO
genome = SeqIO.read(gnm_file[0:len(gnm_file)-3], "genbank")

In [83]:
# Get variants by sample
var = pd.read_csv(datasets[dataset][0],sep='\t')

In [84]:
len(var)

38919

In [85]:
var = var.replace({'FUNCLASS':funclass_translation})

In [86]:
# Changing coordinates to 0-based
var['POS'] = var['POS']-1

In [87]:
# Validation function for checking against genome

def check_against_genome(seqobject,df,pos_base_list,l):
    wrong = []
    slip_sites = dict()
    bad = 0
    good = 0
    pb = df[pos_base_list].to_numpy()
    for item in pb:
        base = seqobject[item[0]:(item[0])+l].seq
        if base != item[1] and len(base) == len(item[1]):
            slip = seqobject[(item[0]-1):(item[0]-1)+l].seq
            if slip == item[1]:
                good += 1
                slip_sites[item[0]] = True
            else:
                bad += 1
                wrong.append([item[1],base,item[0]])
        elif base == item[1] and len(base) == len(item[1]): 
            good += 1
    print ('Total = {}, Wrong = {}, Correct = {}'.format(len(pb),bad,good))
    return(slip_sites,wrong)

In [88]:
check_against_genome(genome,var,['POS','REF'],1)

Total = 38919, Wrong = 0, Correct = 37706


({}, [])

In [89]:
var.head()

,Sample,POS,FILTER,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,min(AF),max(AF),countunique(change),countunique(FUNCLASS),change
0,ERR4859723,65,.,C,T,61,0.934426,2147483647,"0,0,61,0",.,Non-coding,.,.,.,.,.,0.001100,1.000000,1,1,C>T
1,ERR4859723,240,PASS,C,T,99,0.858586,0,"0,0,50,49",.,Non-coding,.,.,.,.,.,0.625000,1.000000,1,1,C>T
2,ERR4859723,444,PASS,T,C,12061,0.985076,0,"10,0,12050,0",LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,gtT/gtC,V60,leader,0.009013,1.000000,2,1,T>C
3,ERR4859723,528,PASS,G,A,22724,0.764654,42,"19,1,11729,10573",LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,ctG/ctA,L88,leader,0.001161,0.764654,3,1,G>A
4,ERR4859723,1986,PASS,A,G,4812,0.926434,0,"4,0,4797,0",LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,tcA/tcG,S394,nsp2,0.001533,1.000000,1,1,A>G


In [90]:
# Variants by site
sites = pd.read_csv(datasets[dataset][1],sep='\t')

In [91]:
len(sites)

5760

In [92]:
sites = sites.replace({'FUNCLASS':funclass_translation})

In [93]:
# Changing coordinates to 0-based
sites['POS'] = sites['POS']-1

In [94]:
check_against_genome(genome,sites,['POS','REF'],1)

Total = 5760, Wrong = 0, Correct = 5017


({}, [])

In [95]:
sites.head()

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,countunique(Sample),min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change
0,20,C,T,.,Non-coding,.,.,.,.,.,1,0.925373,0.925373,ERR4861086,ERR4861086,0.925373,C>T
1,60,G,T,.,Non-coding,.,.,.,.,.,8,0.002343,0.941985,"ERR4860715,ERR4861521","ERR4860715,ERR4860560,ERR4859776,ERR4860474,ER...","0.941985,0.002343,0.006428,0.005267,0.003076,0...",G>T
2,64,T,C,.,Non-coding,.,.,.,.,.,4,0.002558,0.397059,ERR4859832,"ERR4860696,ERR4860829,ERR4859872,ERR4859832","0.012853,0.072581,0.002558,0.397059",T>C
3,65,C,T,.,Non-coding,.,.,.,.,.,129,0.001100,1.000000,"ERR4859723,ERR4859787,ERR4859839,ERR4859870,ER...","ERR4861238,ERR4861207,ERR4861136,ERR4861023,ER...","0.932306,0.914634,0.935213,0.942504,0.948718,0...",C>T
4,66,T,C,.,Non-coding,.,.,.,.,.,1,0.109424,0.109424,ERR4859782,ERR4859782,0.109424,T>C


In [96]:
sel = pd.read_csv(sel_url,sep='\t')

In [97]:
# Add info about sites under sleection to the main variant table
sites = pysqldf('select sites.*, sel.fel,sel.meme,sel.freq from sites left join sel on sites.POS >= sel.pos and sites.POS <= sel.pos+2 ')

In [98]:
sites.head()

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,countunique(Sample),min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change,fel,meme,freq
0,20,C,T,.,Non-coding,.,.,.,.,.,1,0.925373,0.925373,ERR4861086,ERR4861086,0.925373,C>T,NaN,NaN,NaN
1,60,G,T,.,Non-coding,.,.,.,.,.,8,0.002343,0.941985,"ERR4860715,ERR4861521","ERR4860715,ERR4860560,ERR4859776,ERR4860474,ER...","0.941985,0.002343,0.006428,0.005267,0.003076,0...",G>T,NaN,NaN,NaN
2,64,T,C,.,Non-coding,.,.,.,.,.,4,0.002558,0.397059,ERR4859832,"ERR4860696,ERR4860829,ERR4859872,ERR4859832","0.012853,0.072581,0.002558,0.397059",T>C,NaN,NaN,NaN
3,65,C,T,.,Non-coding,.,.,.,.,.,129,0.001100,1.000000,"ERR4859723,ERR4859787,ERR4859839,ERR4859870,ER...","ERR4861238,ERR4861207,ERR4861136,ERR4861023,ER...","0.932306,0.914634,0.935213,0.942504,0.948718,0...",C>T,NaN,NaN,NaN
4,66,T,C,.,Non-coding,.,.,.,.,.,1,0.109424,0.109424,ERR4859782,ERR4859782,0.109424,T>C,NaN,NaN,NaN


In [99]:
voc = pd.read_csv(voc_url, sep='\t',names=['voc_set','mut','position'],header=None)

In [100]:
voc.head()

,voc_set,mut,position
0,B1351,P71L,26454
1,B1351,T205I,28885
2,B1351,K1655N,5227
3,B1351,D80A,21799
4,B1351,D215G,22204


In [101]:
for item in voc['voc_set'].unique():
    sites = pysqldf('select sites.*, mut from sites left join voc on (POS >= position and POS < position+3) and voc_set = "{0}"'.format(item))
    sites = sites.rename(columns={"mut": item})

In [102]:
pysqldf('select * from sites where P1 is not null')

,POS,REF,ALT,IMPACT,FUNCLASS,EFFECT,GENE,CODON,AA,TRID,countunique(Sample),min(AF),max(AF),SAMPLES(above-thresholds),SAMPLES(all),AFs(all),change,fel,meme,freq,B1351,P1,B117,BLOOM
0,3827,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,tCa/tTa,S370L,nsp3,11,0.001676,0.959329,ERR4861281,"ERR4860669,ERR4860428,ERR4860434,ERR4860349,ER...","0.001729,0.002387,0.010461,0.00748,0.001715,0....",C>T,NaN,NaN,NaN,None,S1188L,None,None
1,11288,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,ORF1ab,tCt/tTt,S106F,nsp6,8,0.001122,0.089242,ERR4860024,"ERR4861208,ERR4860881,ERR4859792,ERR4860094,ER...","0.001239,0.001303,0.00131,0.00199,0.089242,0.0...",C>T,2.931933e-03,5.222744e-03,0.000492,None,del,del9,None
2,11289,T,C,LOW,Synonymous,SYNONYMOUS_CODING,ORF1ab,tcT/tcC,S106,nsp6,5,0.001237,0.953882,ERR4860266,"ERR4860266,ERR4861257,ERR4859885,ERR4860791,ER...","0.953882,0.001237,0.002534,0.001995,0.003017",T>C,2.931933e-03,5.222744e-03,0.000492,None,del,del9,None
3,11289,TG,T,HIGH,Non-coding,FRAME_SHIFT,ORF1ab,ggt/,G107,nsp6,5,0.000536,0.076799,ERR4860605,"ERR4860605,ERR4860865,ERR4861473,ERR4861307,ER...","0.076799,5.36E-4,0.001254,0.002835,6.47E-4",TG>T,2.931933e-03,5.222744e-03,0.000492,None,del,del9,None
4,21613,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,S,Ctt/Ttt,L18F,S,745,0.000853,1.000000,"ERR4859724,ERR4859726,ERR4859729,ERR4859733,ER...","ERR4861229,ERR4861212,ERR4861188,ERR4861181,ER...","0.870538,0.853163,0.864213,0.854697,0.879007,0...",C>T,1.859624e-13,6.861178e-13,0.106843,None,L18F,None,None
5,21620,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,S,aCc/aTc,T20I,S,16,0.000918,0.901559,"ERR4859776,ERR4860570,ERR4860886","ERR4861023,ERR4861081,ERR4860570,ERR4860886,ER...","0.001791,0.001719,0.296592,0.301045,0.901559,0...",C>T,6.471967e-03,6.666667e-01,0.002368,None,T20N,None,None
6,21621,C,A,LOW,Synonymous,SYNONYMOUS_CODING,S,acC/acA,T20,S,9,0.000924,0.932692,"ERR4860261,ERR4860269,ERR4860409,ERR4860495,ER...","ERR4860828,ERR4860495,ERR4860409,ERR4860269,ER...","0.003421,0.932692,0.854339,0.923311,0.910961,0...",C>A,6.471967e-03,6.666667e-01,0.002368,None,T20N,None,None
7,21621,C,T,LOW,Synonymous,SYNONYMOUS_CODING,S,acC/acT,T20,S,19,0.000969,0.958603,"ERR4860169,ERR4861536","ERR4860851,ERR4859877,ERR4859767,ERR4859752,ER...","0.020011,0.004739,0.005251,0.00225,0.005557,0....",C>T,6.471967e-03,6.666667e-01,0.002368,None,T20N,None,None
8,21637,C,T,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,S,Cct/Tct,P26S,S,39,0.001191,0.929306,"ERR4859938,ERR4859970,ERR4860050,ERR4860090,ER...","ERR4861106,ERR4860572,ERR4860884,ERR4859778,ER...","0.895203,0.001809,0.001191,0.001526,0.929306,0...",C>T,5.906763e-06,1.451934e-05,0.002831,None,P26S,None,None
9,21973,G,A,MODERATE,Non-synonymous,NON_SYNONYMOUS_CODING,S,Gat/Aat,D138N,S,3,0.001395,0.060357,ERR4860316,"ERR4861298,ERR4860316,ERR4859980","0.001395,0.060357,0.003265",G>A,4.013989e-06,9.851843e-06,0.002613,None,D138Y,None,None


In [103]:
len(sites)

5760

In [104]:
# Assumes df has columns labelled 'ALT' and 'REF'
def chng_type(df):
    df.loc[df['REF'].str.len() == df['ALT'].str.len(), 'type'] = 'SNP'
    df.loc[df['REF'].str.len() != df['ALT'].str.len(), 'type'] = 'Indel'

In [105]:
chng_type(var)
chng_type(sites)

In [106]:
var.to_csv('{}_by_sample_proceseed.tsv'.format(dataset),sep='\t',index=False)
sites.to_csv('{}_by_var_proceseed.tsv'.format(dataset),sep='\t',index=False)
!gzip *.tsv

In [107]:
!ls

bos_by_sample_proceseed.tsv.gz
bos_by_var_proceseed.tsv.gz
cog-post_by_sample_proceseed.tsv.gz
cog-post_by_var_proceseed.tsv.gz
cog-pre_by_sample_proceseed.tsv.gz
cog-pre_by_var_proceseed.tsv.gz
GCF_009858895.2_ASM985889v3_genomic.gbff
GCF_009858895.2_ASM985889v3_genomic.gbff.gz
sample_data


In [108]:
from google.colab import files
files.download('{}_by_sample_proceseed.tsv.gz'.format(dataset))
files.download('{}_by_var_proceseed.tsv.gz'.format(dataset))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>